# Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
import random
import cv2

# Data directories setup

In [ ]:
train_dir = "data/train"
test_dir = "data/test"

# Emotion Categories

In [ ]:
emotion_categories = ["angry", "happy", "neutral"]

# Loading the data

### Training Data

In [ ]:
training_data = []

for emotion in emotion_categories:
    path = os.path.join(train_dir, emotion)
    class_num = emotion_categories.index(emotion)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        training_data.append([img_array, class_num])

### Testing Data

In [ ]:
testing_data = []

for emotion in emotion_categories:
    path = os.path.join(test_dir, emotion)
    class_num = emotion_categories.index(emotion)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        testing_data.append([img_array, class_num])

# Shuffling the data

In [ ]:
random.shuffle(training_data)
random.shuffle(testing_data)

In [ ]:
print(f"training_data : {len(training_data)}, testing_data : {len(testing_data)}")

# Splitting the data into features and labels

### Training Data

In [ ]:
x_train = []
y_train = []

for feature, label in training_data:
    x_train.append(feature)
    y_train.append(label)

### Testing Data

In [ ]:
x_test = []
y_test = []

for feature, label in testing_data:
    x_test.append(feature)
    y_test.append(label)

# Converting the data into numpy arrays

### Training Data

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

### Testing Data

In [ ]:
x_test = np.array(x_test)
y_test = np.array(y_test)

# Reshaping the data

In [ ]:
x_train = x_train.reshape(-1, 48, 48, 1)
x_test = x_test.reshape(-1, 48, 48, 1)

# Normalizing the data

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

# Splitting the data into validation set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Building the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))

# Compiling the model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model Summary

In [ ]:
model.summary()

# Training the model

In [ ]:
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_val, y_val))

# Plotting accuracy and loss (training acc, val acc and training loss, val loss)

In [ ]:
# Retrieve the data from the history object
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create a figure with a specific size
plt.figure(figsize=(12, 6))

# Plot Training and Validation Accuracy
plt.subplot(1, 2, 1)  # (rows, columns, panel number)
plt.plot(history.epoch, train_accuracy, color='blue', label='Train Accuracy')
plt.plot(history.epoch, val_accuracy, color='green', label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()

# Plot Training and Validation Loss
plt.subplot(1, 2, 2)  # (rows, columns, panel number)
plt.plot(history.epoch, train_loss, color='orange', label='Train Loss')
plt.plot(history.epoch, val_loss, color='red', label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()

# Evaluating the model

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Testing the model

In [ ]:
predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

# Plotting test results

In [ ]:
num_images = 8
plt.figure(figsize=(12, 8))

for i in range(num_images):
    true_label = np.argmax(y_test[i])
    predicted_label = predicted_classes[i]

    plt.subplot(1, num_images, i + 1)
    plt.imshow(x_test[i], cmap='gray')
    plt.title(f"True: {emotion_categories[true_label]}\nPred: {emotion_categories[predicted_label]}")
    plt.axis('off')

plt.show()

# Saving the model

In [ ]:
model.save('emotion_detector.h5')